In [5]:
!pip install numpy

In [6]:
!pip install pandas

  Using cached pandas-1.3.5-cp37-cp37m-win_amd64.whl.metadata (12 kB)
   ---------------------------------------- 10.0/10.0 MB 9.7 MB/s eta 0:00:00
   --------------------------------------- 509.2/509.2 kB 10.8 MB/s eta 0:00:00


In [8]:
!pip install matplotlib

     -------------------------------------- 138.5/138.5 kB 2.7 MB/s eta 0:00:00
   ---------------------------------------- 7.2/7.2 MB 10.2 MB/s eta 0:00:00
   --------------------------------------- 965.4/965.4 kB 10.3 MB/s eta 0:00:00
   ---------------------------------------- 55.8/55.8 kB ? eta 0:00:00
   ---------------------------------------- 2.5/2.5 MB 11.5 MB/s eta 0:00:00
   ---------------------------------------- 104.1/104.1 kB ? eta 0:00:00


In [10]:
!pip install seaborn

   ---------------------------------------- 293.3/293.3 kB 4.6 MB/s eta 0:00:00


In [11]:
import os
import pathlib
import sys
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

C:\Users\harit\anaconda3\envs\testenv\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\harit\anaconda3\envs\testenv\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\harit\anaconda3\envs\testenv\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\harit\anaconda3\envs\testenv\lib\site-packages\tensorflow\

In [15]:
!pip install scikit-learn

  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
   ---------------------------------------- 7.1/7.1 MB 6.5 MB/s eta 0:00:00
Using cached joblib-1.3.2-py3-none-any.whl (302 kB)


In [16]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import random

In [19]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Load your dataset
df = pd.read_csv("patientdata.csv")

# Function to bootstrap the data
def bootstrap_data(df, size):
    return df.sample(n=size, replace=True, random_state=42).reset_index(drop=True)

# Function to scale and split data
def prepare_data(df):
    X = df.drop(columns=["outcome"]).values
    y = df["outcome"].values
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    return X_train_scaled, X_val_scaled, y_train, y_val

def run_dl_model(data, hidden_layers):
    X_train, X_val, y_train, y_val = prepare_data(data)
    
    model = Sequential()
    model.add(Dense(4, input_dim=X_train.shape[1], activation='relu'))
    for _ in range(hidden_layers - 1):
        model.add(Dense(4, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])

    start = time.time()
    history = model.fit(
        X_train, y_train,
        epochs=20, batch_size=32,
        verbose=0,
        validation_data=(X_val, y_val)
    )
    end = time.time()

    # Retrieve metrics safely
    hist_keys = history.history.keys()
    print(f"Logged metrics: {hist_keys}")  # Remove this after confirming it works

    # Handle both naming conventions
    train_acc = history.history.get('accuracy', history.history.get('acc'))[-1]
    val_acc = history.history.get('val_accuracy', history.history.get('val_acc'))[-1]
    duration = round(end - start, 2)

    return train_acc, val_acc, duration


# Test sizes and configs
sizes = [1000, 10000, 100000]
results = []

# Run deep learning models
for size in sizes:
    boot_df = bootstrap_data(df, size)
    for layers in [1, 2]:
        train_acc, val_acc, duration = run_dl_model(boot_df, hidden_layers=layers)
        results.append({
            "Data size": size,
            "Hidden layers": layers,
            "Training accuracy": round(train_acc, 4),
            "Validation accuracy": round(val_acc, 4),
            "Execution time (s)": duration
        })

# Output result table
results_df = pd.DataFrame(results)
print(results_df)


Logged metrics: dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])
Logged metrics: dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])
Logged metrics: dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])
Logged metrics: dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])
Logged metrics: dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])
Logged metrics: dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])
   Data size  Hidden layers  Training accuracy  Validation accuracy  \
0       1000              1             0.7812               0.7450   
1       1000              2             0.7762               0.7450   
2      10000              1             0.9965               0.9955   
3      10000              2             0.9969               0.9980   
4     100000              1             0.9988               0.9986   
5     100000              2             0.9985               0.9976   

   Execution time (s)  
0                2.02  
1                2.30  
2               10.37  
3         

In [ ]:
w